# Diagrama ocupaciones 
Este notebook sirve para lograr obtener los distintos subgrupos de ocupaciones que existen según el diagrama del pdf obtenido del INE, de manera que a partir de este diagrama se pueda clasificar las distintas ocupaciones en subgrupos con fuentes oficiales.

## Instalación de dependencias necesarias

In [ ]:
# %pip install camelot-py openpyxl numpy pandas


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Importación de librerías

In [2]:
import camelot
from openpyxl import load_workbook
import pandas as pd
import numpy as np

/home/diggspapu/.pyenv/versions/3.10.16/lib/python3.10/site-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


## Lectura del pdf de diagrama de ocupaciones

In [3]:
# Read the PDF
tables = camelot.read_pdf("./dataset/variables_alternas/DIAGRAMA_OCUPACIONES.pdf", pages="all", flavor="stream")  # or "lattice"
tables.export("./ocupaciones/tables.xlsx", f="excel")  # Export to Excel

/home/diggspapu/.pyenv/versions/3.10.16/lib/python3.10/site-packages/camelot/parsers/base.py:238: UserWarning: No tables found in table area (0, 0, 612.0, 792.0)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)
/home/diggspapu/.pyenv/versions/3.10.16/lib/python3.10/site-packages/camelot/parsers/base.py:238: UserWarning: No tables found in table area (69.3701, 172.4743000000001, 545.6908, 653.1033)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)
/home/diggspapu/.pyenv/versions/3.10.16/lib/python3.10/site-packages/camelot/parsers/base.py:238: UserWarning: No tables found in table area (0, 0, 612.0, 792.0)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)
/home/diggspapu/.pyenv/versions/3.10.16/lib/python3.10/site-packages/camelot/parsers/base.py:238: UserWarning: No tables found in table area (69.37009999999998, 227.85520000000005, 545.9688999999998, 775.7246545454545)
  cols, rows, v_s, h_s = self._generate_colum

## Convertir las tablas a un csv

In [4]:

def clean_table(df):
    df = df.drop([0, 1, 2, 3, 4])
    df = df[df.columns[1:]]
    return df

def fill_row_0(df, row_0):
    df[0] = row_0
    return df

def rellenar_codigos(df):
    # Solo nos interesan las columnas 1 a 4 y la 5 como referencia
    for col in [0, 1, 2, 3, 4]:
        # Solo rellenamos hacia adelante donde la columna 5 tenga valor
        df[col] = df[col].where(df[5].notna()).ffill()
    return df

def cambiar_codigos_por_nombre(df):
    for col in [0, 1, 2, 3, 4]:
        df[col] = df[5].where(df[col].notna()).ffill()
    return df

In [5]:

wb2 = load_workbook('./ocupaciones/tables.xlsx')
dfs = []

In [6]:

for sheet in wb2.sheetnames:
    df = pd.read_excel('./ocupaciones/tables.xlsx', sheet_name=sheet, engine='openpyxl')
    if df.shape[1] == 8:
        df = df.drop([6], axis=1)
        check_df = df.iloc[2]
        if check_df[0] == "Grupo" and check_df[1] == "grupo" and check_df[4] == "Código" and check_df[5] == "Descripción":
            df = clean_table(df)
            dfs.append(df)
    if df.shape[1] == 7:
        check_df = df.iloc[2]
        if check_df[0] == "Grupo" and check_df[1] == "grupo" and check_df[4] == "Código" and check_df[5] == "Descripción":
            df = clean_table(df)
            dfs.append(df)
    del df


In [7]:
temp_df = pd.read_excel('./ocupaciones/tables.xlsx', sheet_name='page-118-table-1')
temp_df = temp_df.drop([0], axis=0)
temp_df = temp_df.drop(["Unnamed: 0"], axis=1)
temp_df = temp_df.rename(columns={0:1, 1:2, 2:3, 3:4, 4:5})
temp_df[0] = np.nan
temp_df = temp_df[[x for x in range(0, 6)]]
last_df = pd.DataFrame(
    columns=[0, 1, 2, 3, 4, 5],
    data=[
        [np.nan, np.nan, np.nan, np.nan, 31009, 'Sargento segundo del ejército, fuerza aérea, marina'],
        [np.nan, np.nan, np.nan, np.nan, 31010, 'Soldado de infantería, ejército ,marina'],
    ]
)
temp_df = pd.concat([temp_df, last_df], ignore_index=True)
dfs.append(temp_df)
df = pd.concat(dfs, ignore_index=False)
df = cambiar_codigos_por_nombre(df)
df = rellenar_codigos(df)
df = df.drop([4], axis=1)
df.to_csv('./ocupaciones/ocupaciones.csv', index=False, sep="|", encoding="utf-8-sig")